In [1]:
using Random
using Statistics
using Distributions
is_in(ci, x) = first(ci) ≤ x ≤ last(ci)

function ci_median(X, Y = similar(X), α = 0.5, A = similar(Y, 10^4))
    for i in eachindex(A)
        sample!(X, Y)
        A[i] = median!(Y)
    end
    quantile!(A, (α/2, 1-α/2))
end

ci_median (generic function with 4 methods)

In [2]:
function sim(; dist = Exponential(), a = median(dist), n = 10, α = 0.5, L = 10^4, M = 10^3)
    c = 0
    X = Vector{Float64}(undef, n)
    Y = similar(X)
    A = similar(Y, M)
    for i in 1:L
        rand!(dist, X)
        ci = ci_median(X, Y, α, A)
        c += is_in(ci, a)
    end
    c/L
end

sim (generic function with 1 method)

In [3]:
@time sim()

  2.872738 seconds (16 allocations: 43.656 KiB)


0.4953

In [4]:
@time sim(n = 20)

  6.591007 seconds (3 allocations: 8.406 KiB)


0.4889

In [5]:
@time sim(dist = Uniform())

  3.027164 seconds (3 allocations: 8.250 KiB)


0.505

In [6]:
@time sim(dist = Normal())

  2.885745 seconds (3 allocations: 8.250 KiB)


0.497

In [7]:
@time sim(dist = Beta(0.1, 0.1), a = 0.5)

  2.894998 seconds (35.70 k allocations: 2.058 MiB, 0.64% compilation time)


0.4938

In [8]:
function sim_bad(; dist = Exponential(), a = median(dist), n = 10, α = 0.5, L = 10^4)
    c = 0
    X = Vector{Float64}(undef, n)
    for i in 1:L
        rand!(dist, X)
        ci = quantile(X, (α/2, 1-α/2))
        c += is_in(ci, a)
    end
    c/L
end

sim_bad (generic function with 1 method)

In [9]:
@time sim_bad()

  0.002594 seconds (10.00 k allocations: 1.526 MiB)


0.8653

In [10]:
@time sim_bad(n = 20)

  0.007316 seconds (10.00 k allocations: 2.289 MiB)


0.9742

In [11]:
@time sim_bad(dist = Uniform())

  0.002878 seconds (10.00 k allocations: 1.526 MiB)


0.8702

In [12]:
@time sim_bad(dist = Normal())

  0.005128 seconds (10.00 k allocations: 1.526 MiB)


0.8541

In [13]:
@time sim_bad(dist = Beta(0.1, 0.1), a = 0.5)

  0.012234 seconds (20.00 k allocations: 2.136 MiB)


0.8721